In [1]:
# !pip uninstall -y trl
# !pip install -q git+https://github.com/huggingface/trl.git
# !pip install -q transformers accelerate peft datasets
# !pip install -U bitsandbytes

In [2]:
import torch
import gc
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification, BitsAndBytesConfig
from peft import LoraConfig, PeftModel
from datasets import load_dataset
from trl.experimental.ppo import PPOConfig, PPOTrainer

/tmp/ipython-input-2905924801.py:6: TRLExperimentalWarning: You are importing from 'trl.experimental'. APIs here are unstable and may change or be removed without notice. Silence this warning by setting environment variable TRL_EXPERIMENTAL_SILENCE=1.
  from trl.experimental.ppo import PPOConfig, PPOTrainer


In [3]:
gc.collect()
torch.cuda.empty_cache()


In [4]:
config = PPOConfig(
    output_dir="./smollm2-ppo-results",
    num_ppo_epochs=1,
    mini_batch_size=8,
    batch_size=64,
    gradient_accumulation_steps=8,
    learning_rate=1.41e-5,
    stop_token="eos",
    response_length=53
)

In [5]:
model_id = "HuggingFaceTB/smollm2-135M-SFT-Only"
reward_model_path = "./smollm2-reward-model-final"

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"
tokenizer.model_max_length = 512

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
policy_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)
policy_model.gradient_checkpointing_enable()

bnb_config_4bit = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
)

peft_config = LoraConfig(
    r=16, lora_alpha=32, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM",
)

value_model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels=1,
    quantization_config=bnb_config_4bit,
    dtype=torch.float16,
    device_map="auto"
)

rm_base = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels=1,
    quantization_config=bnb_config_4bit,
    device_map="auto"
)
reward_model = PeftModel.from_pretrained(rm_base, reward_model_path)
reward_model.eval().requires_grad_(False)



`torch_dtype` is deprecated! Use `dtype` instead!
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at HuggingFaceTB/smollm2-135M-SFT-Only and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at HuggingFaceTB/smollm2-135M-SFT-Only and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): LlamaForSequenceClassification(
      (model): LlamaModel(
        (embed_tokens): Embedding(49152, 576, padding_idx=2)
        (layers): ModuleList(
          (0-29): 30 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=576, out_features=576, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=576, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=576, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
      

In [8]:
dataset = load_dataset("Intel/orca_dpo_pairs", split="train")

def format_prompt(sample):
    prompt = f"<|im_start|>user\n{sample['question']}<|im_end|>\n<|im_start|>assistant\n"
    return {
        "input_ids": tokenizer.encode(
            prompt,
            truncation=True,
            max_length=512
        )
    }

dataset = dataset.select(range(2000)) # Optional: Limit dataset size for speed
dataset = dataset.map(format_prompt, batched=False, remove_columns=dataset.column_names)
eval_dataset = dataset.select(range(20))

In [10]:
print("Initializing Trainer...")
trainer = PPOTrainer(
    args=config,
    processing_class=tokenizer,
    model=policy_model,
    ref_model=None,
    peft_config=peft_config,
    reward_model=reward_model,
    value_model=value_model,
    train_dataset=dataset,
    eval_dataset=eval_dataset
)

Initializing Trainer...


In [11]:
wrapper_class = type(trainer.model)
wrapper_class.gradient_checkpointing_enable = lambda self, **kwargs: self.policy.gradient_checkpointing_enable(**kwargs)
wrapper_class.gradient_checkpointing_disable = lambda self: self.policy.gradient_checkpointing_disable()

In [12]:
print("Starting PPO Training...")
trainer.train()

trainer.save_model("./smollm2-ppo-final")
print("Training Complete!")

Starting PPO Training...
===training policy===


wandb: Currently logged in as: 27100046 (27100046-lahore-university-of-management-sciences) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ query                                          ┃ model response                                 ┃ score         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ user                                           │ [Jaak Pahlej, Ajax Youth Academy, Ajax Ajax,   │ -0.5576171875 │
│ You will be given a definition of a task       │ Ajax Ajax, Ajax Ajax, Ajax Ajax, Ajax Ajax,    │               │
│ first, then some input of the task.            │ Ajax Ajax, Ajax Ajax, Ajax Ajax                │               │
│ This task is about using the specified         │                                                │               │
│ sentence and converting the sentence to        │                                                │               │
│ Resource Description Framework (RDF) triplets  │                                                │               │
│ of the form (subject, predicate object). The   │                                                │               │
│ RDF triplets generated must be such that the   │                                                │               │
│ triplets accurately capture the structure and  │                                                │               │
│ semantics of the input sentence. The input is  │                                                │               │
│ a sentence and the output is a list of         │                                                │               │
│ triplets of the form  that capture the         │                                                │               │
│ relationships present in the sentence. When a  │                                                │               │
│ sentence has more than 1 RDF triplet possible, │                                                │               │
│ the output must contain all of them.           │                                                │               │
│                                                │                                                │               │
│ AFC Ajax (amateurs)'s ground is Sportpark De   │                                                │               │
│ Toekomst where Ajax Youth Academy also play.   │                                                │               │
│ Output:                                        │                                                │               │
│ assistant                                      │                                                │               │
│                                                │                                                │               │
├────────────────────────────────────────────────┼────────────────────────────────────────────────┼───────────────┤
│ user                                           │ Midsummer House is a moderately priced         │ -1.681640625  │
│ Generate an approximately fifteen-word         │ restaurant located in the heart of the city,   │               │
│ sentence that describes all this data:         │ offering a delightful blend of Chinese cuisine │               │
│ Midsummer House eatType restaurant; Midsummer  │ and a cozy atmosphere. The restaurant is known │               │
│ House food Chinese; Midsummer House priceRange │ for its exceptional service, and the customer  │               │
│ moderate; Midsummer House customer rating 3    │ rating is generally favorable, with a 3 out of │               │
│ out of 5; Midsummer House near All Bar One     │                                                │               │
│ assistant                                      │                                                │               │
│                                                │                                                │               │
├────────────────────────────────────────────────┼──────

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ query                                         ┃ model response                               ┃ score            ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ user                                          │ [Jaakamp, Ajax Youth Academy, Ajax Ajax,     │ -0.1483154296875 │
│ You will be given a definition of a task      │ Ajax Ajax, Ajax Ajax, Ajax Ajax, Ajax Ajax,  │                  │
│ first, then some input of the task.           │ Ajax Ajax, Ajax Ajax, Ajax Ajax, Ajax        │                  │
│ This task is about using the specified        │                                              │                  │
│ sentence and converting the sentence to       │                                              │                  │
│ Resource Description Framework (RDF) triplets │                                              │                  │
│ of the form (subject, predicate object). The  │                                              │                  │
│ RDF triplets generated must be such that the  │                                              │                  │
│ triplets accurately capture the structure and │                                              │                  │
│ semantics of the input sentence. The input is │                                              │                  │
│ a sentence and the output is a list of        │                                              │                  │
│ triplets of the form  that capture the        │                                              │                  │
│ relationships present in the sentence. When a │                                              │                  │
│ sentence has more than 1 RDF triplet          │                                              │                  │
│ possible, the output must contain all of      │                                              │                  │
│ them.                                         │                                              │                  │
│                                               │                                              │                  │
│ AFC Ajax (amateurs)'s ground is Sportpark De  │                                              │                  │
│ Toekomst where Ajax Youth Academy also play.  │                                              │                  │
│ Output:                                       │                                              │                  │
│ assistant                                     │                                              │                  │
│                                               │                                              │                  │
├───────────────────────────────────────────────┼──────────────────────────────────────────────┼──────────────────┤
│ user                                          │ Midsummer House is a moderately priced       │ -1.681640625     │
│ Generate an approximately fifteen-word        │ restaurant located in the heart of the city, │                  │
│ sentence that describes all this data:        │ offering a delightful blend of Chinese       │                  │
│ Midsummer House eatType restaurant; Midsummer │ cuisine and a cozy atmosphere. The           │                  │
│ House food Chinese; Midsummer House           │ restaurant is known for its exceptional      │                  │
│ priceRange moderate; Midsummer House customer │ service, and the customer rating is          │                  │
│ rating 3 out of 5; Midsummer House near All   │ generally favorable, with a 3 out of         │                  │
│ Bar One                                       │                                              │                  │
│ assistant                                     │       

/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ query                                         ┃ model response                               ┃ score            ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ user                                          │ [Jaakamp, Ajax Youth Academy, Ajax Ajax,     │ -0.1483154296875 │
│ You will be given a definition of a task      │ Ajax Ajax, Ajax Ajax, Ajax Ajax, Ajax Ajax,  │                  │
│ first, then some input of the task.           │ Ajax Ajax, Ajax Ajax, Ajax Ajax, Ajax        │                  │
│ This task is about using the specified        │                                              │                  │
│ sentence and converting the sentence to       │                                              │                  │
│ Resource Description Framework (RDF) triplets │                                              │                  │
│ of the form (subject, predicate object). The  │                                              │                  │
│ RDF triplets generated must be such that the  │                                              │                  │
│ triplets accurately capture the structure and │                                              │                  │
│ semantics of the input sentence. The input is │                                              │                  │
│ a sentence and the output is a list of        │                                              │                  │
│ triplets of the form  that capture the        │                                              │                  │
│ relationships present in the sentence. When a │                                              │                  │
│ sentence has more than 1 RDF triplet          │                                              │                  │
│ possible, the output must contain all of      │                                              │                  │
│ them.                                         │                                              │                  │
│                                               │                                              │                  │
│ AFC Ajax (amateurs)'s ground is Sportpark De  │                                              │                  │
│ Toekomst where Ajax Youth Academy also play.  │                                              │                  │
│ Output:                                       │                                              │                  │
│ assistant                                     │                                              │                  │
│                                               │                                              │                  │
├───────────────────────────────────────────────┼──────────────────────────────────────────────┼──────────────────┤
│ user                                          │ Midsummer House is a moderately priced       │ -1.681640625     │
│ Generate an approximately fifteen-word        │ restaurant located in the heart of the city, │                  │
│ sentence that describes all this data:        │ offering a delightful blend of Chinese       │                  │
│ Midsummer House eatType restaurant; Midsummer │ cuisine and a cozy atmosphere. The           │                  │
│ House food Chinese; Midsummer House           │ restaurant is known for its exceptional      │                  │
│ priceRange moderate; Midsummer House customer │ service, and the customer rating is          │                  │
│ rating 3 out of 5; Midsummer House near All   │ generally favorable, with a 3 out of         │                  │
│ Bar One                                       │                                              │                  │
│ assistant                                     │       

/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Training Complete!


In [13]:
from google.colab import files
import shutil

shutil.make_archive('smollm2-ppo-final', 'zip', './smollm2-ppo-final')

files.download('smollm2-ppo-final.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>